## Algoritmo de optimización



In [26]:
import json
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from datetime import datetime
import uuid

# Constantes del problema
k = 0.14
n = 0.02
TDS_min = 0.05
TDS_max = 2
CTI = 0.3  # 300 ms
alpha1 = 1
alpha2 = 2
beta2 = 100

# Valores por defecto para usar cuando falten datos
DEFAULT_PICKUP = 0.05  # Valor mínimo típico para pickup
DEFAULT_ISHC = 1.0     # Valor típico para corriente de cortocircuito
DEFAULT_TDS = 0.1      # Valor inicial razonable para TDS

# Función para calcular el tiempo de operación
def calculate_operation_time(Ishc, Ipi, TDSi):
    """Calcula el tiempo de operación del relé"""
    # Verificar valores válidos para evitar errores
    if Ishc is None or Ipi is None or TDSi is None or Ishc == 0 or Ipi == 0:
        return 999.0  # Valor muy alto para indicar que no es una solución viable
    
    try:
        result = (k / ((Ishc/Ipi)**n - 1)) * TDSi
        # Verificar que el resultado sea un número válido
        if np.isnan(result) or np.isinf(result) or result <= 0:
            return 999.0
        return result
    except Exception:
        # Capturar cualquier error matemático (división por cero, etc.)
        return 999.0

# Función objetivo
def objective_function(tds_values, relay_pairs):
    """
    Implementa la función objetivo:
    OF = α₁∑(t_i)² + α₂∑(Δt_mbj - β₂(Δt_mbj - |Δt_mbj|))²
    """
    total_cost = 0
    coordination_penalties = 0
    
    # Para cada par de relés
    for i, pair in enumerate(relay_pairs):
        main_relay = pair['main_relay']
        backup_relay = pair['backup_relay']
        
        # Saltear pares que tienen valores Time_out = 0 en el respaldo
        if backup_relay.get('Time_out', 0) == 0:
            continue
        
        # Índices en el vector de decisión
        main_idx = i * 2
        backup_idx = i * 2 + 1
        
        # Obtener valores seguros, con manejo de None/null
        main_ishc = main_relay.get('Ishc', DEFAULT_ISHC)
        main_pickup = main_relay.get('pick_up', DEFAULT_PICKUP)
        if main_pickup is None:
            main_pickup = DEFAULT_PICKUP
            
        backup_ishc = backup_relay.get('Ishc', DEFAULT_ISHC)
        backup_pickup = backup_relay.get('pick_up', DEFAULT_PICKUP)
        if backup_pickup is None:
            backup_pickup = DEFAULT_PICKUP
        
        # Cálculo de tiempos de operación con los nuevos TDS
        if main_idx < len(tds_values) and backup_idx < len(tds_values):
            main_time = calculate_operation_time(
                main_ishc, 
                main_pickup, 
                tds_values[main_idx]
            )
            
            backup_time = calculate_operation_time(
                backup_ishc, 
                backup_pickup, 
                tds_values[backup_idx]
            )
            
            # Primer término - suma de tiempos de operación al cuadrado
            total_cost += alpha1 * (main_time**2)
            
            # Segundo término - penalización por violación de coordinación
            delta_t = backup_time - main_time
            penalty = delta_t - beta2 * (delta_t - abs(delta_t))
            coordination_penalties += alpha2 * (penalty**2)
            
            # Verificar que el CTI se cumpla
            if delta_t < CTI:
                coordination_penalties += 1000 * (CTI - delta_t)**2
    
    return total_cost + coordination_penalties

# Restricciones para la optimización
def tds_constraints(relay_pairs):
    # Restricciones para TDS_min <= TDS <= TDS_max
    constraints = []
    
    for i in range(len(relay_pairs) * 2):  # Cada par tiene 2 relés
        # Restricción inferior: TDS >= TDS_min
        constraints.append({
            'type': 'ineq',
            'fun': lambda x, idx=i: x[idx] - TDS_min
        })
        
        # Restricción superior: TDS <= TDS_max
        constraints.append({
            'type': 'ineq',
            'fun': lambda x, idx=i: TDS_max - x[idx]
        })
    
    return constraints

# Preprocesamiento para eliminar pares inválidos o corregirlos
def preprocess_relay_pairs(relay_pairs):
    valid_pairs = []
    
    for pair in relay_pairs:
        # Revisar si es un par válido
        main_relay = pair['main_relay']
        backup_relay = pair['backup_relay']
        
        # Si el tiempo de operación del respaldo es 0, necesitamos asignar valores predeterminados
        if backup_relay.get('Time_out', 0) == 0:
            # Caso especial - asignar valores predeterminados
            if backup_relay.get('pick_up') is None:
                backup_relay['pick_up'] = DEFAULT_PICKUP
            if backup_relay.get('TDS') is None:
                backup_relay['TDS'] = DEFAULT_TDS
            if backup_relay.get('Ishc') == 0:
                backup_relay['Ishc'] = main_relay.get('Ishc', DEFAULT_ISHC)
        
        # Si el relé principal tiene valores faltantes o nulos, asignarle valores predeterminados
        if main_relay.get('pick_up') is None:
            main_relay['pick_up'] = DEFAULT_PICKUP
        if main_relay.get('TDS') is None:
            main_relay['TDS'] = DEFAULT_TDS
        if main_relay.get('Ishc', 0) == 0:
            main_relay['Ishc'] = DEFAULT_ISHC
            
        valid_pairs.append(pair)
    
    return valid_pairs

# Procesamiento principal
def optimize_relay_settings(input_data):
    # Organizar datos por escenario
    scenarios = {}
    for pair in input_data:
        scenario_id = pair['scenario_id']
        if scenario_id not in scenarios:
            scenarios[scenario_id] = []
        scenarios[scenario_id].append(pair)
    
    # Resultados finales para todos los escenarios
    all_results = []
    
    # Optimizar cada escenario
    for scenario_id, relay_pairs in scenarios.items():
        print(f"Procesando escenario: {scenario_id}")
        
        # Preprocesar los pares de relés para manejar casos especiales
        valid_relay_pairs = preprocess_relay_pairs(relay_pairs)
        
        if not valid_relay_pairs:
            print(f"No hay pares válidos para el escenario {scenario_id}")
            continue
        
        # Inicializar con los valores TDS actuales como punto de partida
        initial_tds = []
        for pair in valid_relay_pairs:
            # Usar valores predeterminados si son None
            main_tds = pair['main_relay'].get('TDS')
            if main_tds is None:
                main_tds = DEFAULT_TDS
                
            backup_tds = pair['backup_relay'].get('TDS')
            if backup_tds is None:
                backup_tds = DEFAULT_TDS
                
            initial_tds.append(main_tds)
            initial_tds.append(backup_tds)
        
        # Definir restricciones
        constraints = tds_constraints(valid_relay_pairs)
        
        try:
            # Optimizar
            result = minimize(
                objective_function,
                initial_tds,
                args=(valid_relay_pairs,),
                method='SLSQP',
                constraints=constraints,
                options={'maxiter': 500, 'disp': False}
            )
            
            # Organizar resultados
            relay_values = {}
            for i, pair in enumerate(valid_relay_pairs):
                # Agregar valores optimizados de los relés principales
                main_relay = pair['main_relay']['relay']
                if main_relay not in relay_values and i*2 < len(result.x):
                    relay_values[main_relay] = {
                        "TDS": round(result.x[i*2], 5),
                        "pickup": pair['main_relay']['pick_up']
                    }
                
                # Agregar valores optimizados de los relés de respaldo
                backup_relay = pair['backup_relay']['relay']
                if backup_relay not in relay_values and i*2+1 < len(result.x):
                    relay_values[backup_relay] = {
                        "TDS": round(result.x[i*2+1], 5),
                        "pickup": pair['backup_relay']['pick_up']
                    }
            
            # Crear documento de resultados
            scenario_result = {
                "_id": {"$oid": str(uuid.uuid4()).replace("-", "")[:24]},
                "scenario_id": scenario_id,
                "timestamp": datetime.now().isoformat(),
                "relay_values": relay_values
            }
            
            all_results.append(scenario_result)
            print(f"Optimización exitosa para el escenario {scenario_id}")
            
        except Exception as e:
            print(f"Error en la optimización del escenario {scenario_id}: {e}")
            # Crear un resultado con los valores iniciales en caso de error
            relay_values = {}
            for i, pair in enumerate(valid_relay_pairs):
                main_relay = pair['main_relay']['relay']
                if main_relay not in relay_values:
                    relay_values[main_relay] = {
                        "TDS": pair['main_relay'].get('TDS', DEFAULT_TDS),
                        "pickup": pair['main_relay'].get('pick_up', DEFAULT_PICKUP)
                    }
                
                backup_relay = pair['backup_relay']['relay']
                if backup_relay not in relay_values:
                    relay_values[backup_relay] = {
                        "TDS": pair['backup_relay'].get('TDS', DEFAULT_TDS),
                        "pickup": pair['backup_relay'].get('pick_up', DEFAULT_PICKUP)
                    }
            
            # Crear documento de resultados con valores iniciales
            scenario_result = {
                "_id": {"$oid": str(uuid.uuid4()).replace("-", "")[:24]},
                "scenario_id": scenario_id,
                "timestamp": datetime.now().isoformat(),
                "relay_values": relay_values,
                "error": str(e)
            }
            
            all_results.append(scenario_result)
    
    return all_results

# Cargar datos de entrada
with open('/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs_automation.json', 'r') as file:
    input_data = json.load(file)

# Ejecutar la optimización
optimized_settings = optimize_relay_settings(input_data)

# Guardar resultados
with open('/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/raw/optimized_relay_values_paper.json', 'w') as file:
    json.dump(optimized_settings, file, indent=2)

print(f"Optimización completada para {len(optimized_settings)} escenarios.")


Procesando escenario: scenario_1
Optimización exitosa para el escenario scenario_1
Procesando escenario: scenario_2
Optimización exitosa para el escenario scenario_2
Procesando escenario: scenario_3
Optimización exitosa para el escenario scenario_3
Procesando escenario: scenario_4
Optimización exitosa para el escenario scenario_4
Procesando escenario: scenario_5
Optimización exitosa para el escenario scenario_5
Procesando escenario: scenario_6
Optimización exitosa para el escenario scenario_6
Procesando escenario: scenario_7
Optimización exitosa para el escenario scenario_7
Procesando escenario: scenario_8
Optimización exitosa para el escenario scenario_8
Procesando escenario: scenario_9
Optimización exitosa para el escenario scenario_9
Procesando escenario: scenario_10
Optimización exitosa para el escenario scenario_10
Procesando escenario: scenario_11
Optimización exitosa para el escenario scenario_11
Procesando escenario: scenario_12
Optimización exitosa para el escenario scenario_1